<a href="https://colab.research.google.com/github/TheOneTrueGuy/CASI/blob/main/Casi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cyclical Adverserial Stepwise Improvement
further explanation follows the code cells below

In [ ]:
# for Gemini
!pip install gradio google-generativeai

In [ ]:
#for openai
!pip install gradio openai python-dotenv

In [ ]:
# openai version
import gradio as gr
import openai
import os, json


from google.colab import userdata
key=userdata.get("OPENAI_API_KEY")
openai.api_key = key #os.getenv("OPENAI_API_KEY")  # Replace with a key or use a secret

# Function to create a response from the LLM
def generate_response(model, prompt, input_text, critique=None):
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": input_text}
    ]
    if critique:
        messages.append({"role": "assistant", "content": critique})

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        max_tokens=150,
        temperature=0.7
    )
    return response.choices[0].message.content

# Generator function
def generator(prompt, user_input, critic_feedback):
    # Adjust the prompt to expect a JSON response
    json_prompt = f"{prompt}. Please respond in JSON format with keys for 'response' and 'suggestions'."

    full_prompt = f"""
    {json_prompt}
    User input: {user_input}
    Critique: {critic_feedback}
    """

    raw_response = generate_response("local_model", full_prompt, "")
    try:
        json_response = json.loads(raw_response)
        return json_response['response'], json_response.get('suggestions', [])
    except json.JSONDecodeError:
        # In case the model doesn't return valid JSON, handle the error gracefully
        return raw_response, []

# Critic function
def critic(prompt, generator_output, suggestions=None):
    # If suggestions are provided, include them in the prompt
    suggestions_str = ", ".join(suggestions) if suggestions else "No suggestions provided."
    json_prompt = f"{prompt}. Incorporate these suggestions in your analysis: {suggestions_str}"

    full_prompt = f"""
    {json_prompt}
    Generator's Output: {generator_output}
    """

    raw_response = generate_response("local_model", full_prompt, "")
    try:
        json_response = json.loads(raw_response)
        return json_response['response'], json_response.get('suggestions', [])
    except json.JSONDecodeError:
        return raw_response, []

# Gradio interface setup
with gr.Blocks() as demo:
    with gr.Row():  # Two-column layout
        with gr.Column():
            gen_prompt = gr.Textbox(label="Generator System Prompt", lines=3, value="Formalize and expand this idea")
            gen_input = gr.Textbox(label="Generator Input", lines=2)
            gen_output = gr.Textbox(label="Generator Output", lines=5)
            gen_suggestions = gr.Textbox(label="Generator Suggestions", lines=2)
            gen_submit = gr.Button("Submit Generator")

        with gr.Column():
            crit_prompt = gr.Textbox(label="Critic System Prompt", lines=3, value="You are a constructive critic, analyze and critique this idea")
            crit_input = gr.Textbox(label="Critic Input (Optional)", lines=2)
            crit_output = gr.Textbox(label="Critic Output", lines=5)
            crit_suggestions = gr.Textbox(label="Critic Suggestions", lines=2)
            crit_submit = gr.Button("Submit Critic")

    def on_gen_submit(gen_prompt, gen_input, crit_output):
        output, suggestions = generator(gen_prompt, gen_input, crit_output)
        new_gen_prompt = "Using your brilliant imagination and knowledge, answer these criticisms step-by-step with new ideas that correct or fulfill each criticism."
        return {gen_output: output, gen_prompt: new_gen_prompt, gen_suggestions: str(suggestions)}

    def on_crit_submit(crit_prompt, crit_input, gen_output, gen_suggestions):
        output, suggestions = critic(crit_prompt, gen_output, eval(gen_suggestions))  # Assuming suggestions are stored as a string of list
        new_crit_prompt = "Analyze the revised idea, focusing on new aspects introduced by the generator and provide further critique."
        return {crit_output: output, crit_prompt: new_crit_prompt, crit_suggestions: str(suggestions)}

    # Event handlers for the buttons
    gen_submit.click(
        fn=on_gen_submit,
        inputs=[gen_prompt, gen_input, crit_output],
        outputs=[gen_output, gen_prompt, gen_suggestions]
    )
    crit_submit.click(
        fn=on_crit_submit,
        inputs=[crit_prompt, crit_input, gen_output, gen_suggestions],
        outputs=[crit_output, crit_prompt, crit_suggestions]
    )

# Launch the interface
demo.launch(debug=True)

In [9]:
#Cyclical Adversarail Stepwise Improvement
import gradio as gr
import google.generativeai as genai
import os, json


from google.colab import userdata
api_key =userdata.get("GEMINI_API_KEY")


# Configure the Gemini API client
genai.configure(api_key=api_key)

# Create a generative model object
model = genai.GenerativeModel("gemini-1.5-flash")  # Replace with your model name if different

def generate_response(prompt, input_text, critique=None):
    # Construct the prompt content
    full_content = prompt
    if input_text:
        full_content += f"\n\nUser input: {input_text}"
    if critique:
        full_content += f"\n\nPrevious critique: {critique}"

    # Send request to generate content
    response = model.generate_content(full_content)
    predicted_output = response.text

    try:
        json_response = json.loads(predicted_output)
        return json_response['response'], json_response.get('suggestions', [])
    except json.JSONDecodeError:
        # In case the model doesn't return valid JSON, handle the error gracefully
        return predicted_output, []

def generator(prompt, user_input, critic_feedback):
  # Adjust the prompt to expect a JSON response
  json_prompt = f"{prompt}. Please respond in JSON format with keys for 'response' and 'suggestions'."

  full_prompt = f"""
  {json_prompt}
  User input: {user_input}
  Critique: {critic_feedback}
  """

  raw_response, suggestions = generate_response(full_prompt, "")
  return raw_response, suggestions

def critic(prompt, generator_output, suggestions=None):
  # If suggestions are provided, include them in the prompt
  suggestions_str = ", ".join(suggestions) if suggestions else "No suggestions provided."
  json_prompt = f"{prompt}. Incorporate these suggestions in your analysis: {suggestions_str}"

  full_prompt = f"""
  {json_prompt}
  Generator's Output: {generator_output}
  """

  raw_response, suggestions = generate_response(full_prompt, "")
  return raw_response, suggestions

with gr.Blocks() as demo:
    with gr.Row():  # Two-column layout
        with gr.Column():
            gen_prompt = gr.Textbox(label="Generator System Prompt", lines=3, value="Formalize and expand this idea")
            gen_input = gr.Textbox(label="Generator Input", lines=2)
            gen_output = gr.Textbox(label="Generator Output", lines=5)
            gen_suggestions = gr.Textbox(label="Generator Suggestions", lines=2)
            gen_submit = gr.Button("Submit Generator")

        with gr.Column():
            crit_prompt = gr.Textbox(label="Critic System Prompt", lines=3, value="You are a constructive critic, analyze and critique this idea")
            crit_input = gr.Textbox(label="Critic Input (Optional)", lines=2)
            crit_output = gr.Textbox(label="Critic Output", lines=5)
            crit_suggestions = gr.Textbox(label="Critic Suggestions", lines=2)
            crit_submit = gr.Button("Submit Critic")

    def on_gen_submit(gen_prompt, gen_input, crit_output):
        output, suggestions = generator(gen_prompt, gen_input, crit_output)
        new_gen_prompt = "Using your brilliant imagination and knowledge, answer these criticisms step-by-step with new ideas that correct or fulfill each criticism."
        return output, new_gen_prompt, str(suggestions)

    def on_crit_submit(crit_prompt, crit_input, gen_output, gen_suggestions):
        output, suggestions = critic(crit_prompt, gen_output, eval(gen_suggestions))
        new_crit_prompt = "Analyze the revised idea, focusing on new aspects introduced by the generator and provide further critique."
        return output, new_crit_prompt, str(suggestions)

    # Event handlers INSIDE the Blocks context
    gen_submit.click(
        fn=on_gen_submit,
        inputs=[gen_prompt, gen_input, crit_output],
        outputs=[gen_output, gen_prompt, gen_suggestions]
    )
    crit_submit.click(
        fn=on_crit_submit,
        inputs=[crit_prompt, crit_input, gen_output, gen_suggestions],
        outputs=[crit_output, crit_prompt, crit_suggestions]
    )

# Launch the interface
demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ae6ca76bb32dee75e0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://ae6ca76bb32dee75e0.gradio.live


"Cyclical Adversarial Stepwise Improvement" is an interesting approach to iterative content creation or problem-solving through AI. Here's a breakdown of the workflow, logic, and interface:

### Interface Design:

- **Two Columns Layout:**
  - **Left Column (Generator)**:
    - **Top Text Box**: System Prompt for Generator - This could contain instructions for the generator on how to interpret or respond to the input.
    - **Middle Text Box**: User Input for Generator - Where users type what they want the generator to create or solve.
    - **Bottom Text Box**: Generator's Output - Displays the response or creation from the generator.

  - **Right Column (Critic)**:
    - **Top Text Box**: System Prompt for Critic - Instructions on how to critique or improve upon the generator's output.
    - **Middle Text Box**: User Input for Critic - Optional for additional critique parameters or context.
    - **Bottom Text Box**: Critic's Output - Shows the feedback or critique from the critic AI.

- **Buttons**:
  - Each column has a **Submit** button which triggers the processing of inputs from both sides.

### Workflow and Logic:

1. **Initial Setup:**
   - Users can set up the system prompts in both generator and critic columns. These prompts should guide the AI on how to generate and critique content respectively.

2. **First Cycle:**
   - User enters an input in the Generator's middle text box.
   - User clicks "Submit" on the Generator side:
     - The system sends:
       - Generator's system prompt
       - User's input to the Generator
       - The previous output from the Critic (if any, otherwise it starts from scratch)
     - The LLM processes this and generates content, which appears in the Generator's output box.

3. **Critique Cycle:**
   - After the Generator produces an output, the user might adjust or input something in the Critic's middle box or leave it blank for general critique.
   - User clicks "Submit" on the Critic side:
     - Sends:
       - Critic's system prompt
       - Content from the Generator's output
       - Any additional user input in the Critic's middle box
     - The LLM provides critique or suggestions, which appear in the Critic's output box.

4. **Iterative Improvement:**
   - Each subsequent cycle involves:
     - Reading the critique in the Critic's output.
     - Possibly adjusting the Generator's input based on the critique.
     - Submitting again to see improvements or iterations on the original task.

5. **Feedback Loop:**
   - Both the Generator and Critic can include in their outputs suggestions for how to better frame the next interaction or critique, fostering a learning loop.

### Technical Implementation:

- **Backend:**
  - Use an API to communicate with your LLM service. You'll need to handle the state management for each cycle, storing previous inputs and outputs.

- **Frontend:**
  - Implement a responsive two-column layout using frameworks like React, Vue, or Angular. Ensure state management libraries like Redux or Vuex are used for handling the dynamic updates to the text boxes.

- **API Calls:**
  - When a button is clicked, make an asynchronous call to your backend which then queries the LLM. Ensure you handle the responses so they populate the correct output boxes.

- **Security & Privacy:**
  - Since you're dealing with user inputs and AI outputs, ensure you implement proper data handling, security practices, and potentially user authentication if the service needs to be personalized or secure.

This setup would allow for a dynamic, interactive environment where users can see tangible improvements in AI-generated content or solutions through an iterative process of generation and critique.